In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np

import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from torch.utils.data import Dataset, DataLoader
from pickle import dump
from pickle import load

import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# Load the dataset _Mix
# X = np.load('EBR32094107_BOT_Head100_x.npy', allow_pickle=True)  
## X = X[:, :15] 으로 학습 시, Epoch [31/100000], Train Loss: 0.00883, Train Acc: 0.911, Valid Loss: 0.02390, Valid Acc: 0.902
# y = np.load('All_Head_Simple_NoMix_y.npy')
# category_var = np.load('EBR32094107_BOT_Head100_y.npy', allow_pickle=True)

# X = np.load('All_Head_Simple_NoMix_x.npy')  # 여기 데이터셋으로 검증하면 정확도 0.89 임
# category_var = np.load('All_Head_Simple_NoMix_y.npy')

# X = np.load('All_Head_Simple_x.npy')
# category_var = np.load('All_Head_Simple_y.npy')

# X = np.load('D:/py/DGS/EBR32094107_BOT_Head66_x.npy', allow_pickle=True)
# category_var = np.load('D:/py/DGS/EBR32094107_BOT_Head66_y.npy', allow_pickle=True)

# X = np.load('D:/py/DGS/EBR32094107_BOT_Head_NoMix_x.npy', allow_pickle=True)
# category_var = np.load('D:/py/DGS/EBR32094107_BOT_Head_NoMix_y.npy', allow_pickle=True)

######################################################
#### 학습 / 검증용 #########
X = np.load('Head_230927_1_x.npy', allow_pickle=True)
# X = X[ : , :14]  # 테스트용 동일 모델에서는 옵션값만 있으면 학습이되는지 확인하기 위함 : 결과) 동일 모델로 구성된 데이터셋에서 부품정보없이, 옵션정보만 입력해도 예측율 동일 수준 임

# category_var = np.load('Head_230921_2_y.npy', allow_pickle=True)
# print(category_var[:10])


X = X.astype(np.float)
# category_var = category_var.astype(np.int32)

# num_category = len(np.unique(category_var))+1 ## 유니크 범주 개수
# identity_mat = np.eye(num_category) ## 단위 행렬
# y = identity_mat[category_var] ## 범주에 대응하는 행 추출

y = np.load('Head_230927_1_y.npy', allow_pickle=True)
y = y.astype(np.int32)
print(X.shape , y.shape)
X_train, X_test, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

le = LabelEncoder()
# scaler = StandardScaler()
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_test)
##########################################################################


##########################################################################
######## Test용 ############
# X = np.load('Head_230921_2_test_x.npy', allow_pickle=True)
# # category_var = np.load('Head_230921_2_test_y.npy', allow_pickle=True)

# X = X.astype(np.int32)
# # category_var = category_var.astype(np.int)

# # num_category = len(np.unique(category_var))+1 ## 유니크 범주 개수
# # identity_mat = np.eye(num_category) ## 단위 행렬
# # y = identity_mat[category_var] ## 범주에 대응하는 행 추출

# y = np.load('Head_230921_2_test_y.npy', allow_pickle=True)
# y = y.astype(np.int32)
# print(X.shape , y.shape)
# X_train, X_test, y_train, y_valid = train_test_split(X, y, test_size=0.99, random_state=42)

# le = LabelEncoder()
# # scaler = StandardScaler()
# # scaler = MinMaxScaler()
# scaler = load(open('MinMax_scaler_Head_230921.pkl', 'rb'))    #Scaler 불러오기

# ## X_train_scaled = scaler.fit_transform(X_train)    # Test 시, 사용하면 안됨
# X_valid_scaled = scaler.transform(X_test)
# print(X_valid_scaled.shape)
###########################################################################


##########################################################################
######## NoTrain 용 ############
# X = np.load('Head_230921_2_NoTrain_x.npy', allow_pickle=True)
# # category_var = np.load('Head_230921_2_NoTrain_y.npy', allow_pickle=True)

# X = X.astype(np.int32)
# # category_var = category_var.astype(np.int)

# # num_category = len(np.unique(category_var))+1 ## 유니크 범주 개수
# # identity_mat = np.eye(num_category) ## 단위 행렬
# # y = identity_mat[category_var] ## 범주에 대응하는 행 추출

# y = np.load('Head_230921_2_NoTrain_y.npy', allow_pickle=True)
# y = y.astype(np.int32)
# print(X.shape , y.shape)
# X_test = X
# y_valid = y

# le = LabelEncoder()
# # scaler = StandardScaler()
# # scaler = MinMaxScaler()
# scaler = load(open('MinMax_scaler_Head_230921.pkl', 'rb'))    #Scaler 불러오기

# X_valid_scaled = scaler.transform(X_test)
# # print(X_valid_scaled.shape)
###########################################################################

inputsize = X.shape[1]
num_classes = len(set(y[0]))
outsize = len(y[0])

print(inputsize , num_classes , outsize)


(1632, 9014) (1632, 1000)
9014 9 1000


In [5]:
# 데이터셋 클래스 정의
class AssemblyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [6]:
batch_size = 32
train_dataset = AssemblyDataset(X_train_scaled, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataset = AssemblyDataset(X_valid_scaled, y_valid)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [7]:
# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers , outsize, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers , batch_first=True)
        self.fc = nn.Linear(hidden_size, outsize*num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
#         out = out[:, -1, :]  # 마지막 시퀀스 스텝의 출력만 사용
        out = self.fc(out)
        out = out.view(-1, outsize , num_classes)
        return out

In [8]:
# 모델 생성

device = torch.device("cuda:0")
input_size = inputsize
hidden_size = 1000
num_layers = 2
num_classes = num_classes
model = LSTMModel(input_size, hidden_size, num_layers , outsize , num_classes).to(device)

# 손실 함수와 최적화 알고리즘 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



In [9]:
# 학습 루프
num_epochs = 100000
acc_compare = 0

for epoch in range(num_epochs):
    model.train()

    total_loss = 0
    correct_t = 0
    total_t = 0

    for batch_input, batch_output_t in train_loader:
        batch_input = batch_input.to(device)
        batch_output_t = batch_output_t.to(device)
        optimizer.zero_grad()

        # 순방향 전파
        outputs = model(batch_input.unsqueeze(1))  # LSTM 입력 형태로 변환

        # 손실 계산
        # print(outputs.view(-1 , 10).shape, batch_output_t.view(-1).shape , batch_output_t.shape)
        loss = criterion(outputs.view(-1 , num_classes), batch_output_t.view(-1) )  # 모양을 맞춤
        total_loss += loss.item()

        # 역전파 및 가중치 업데이트
        loss.backward()
        optimizer.step()
       
        predicted_t = outputs.argmax(dim=2)
#         print(predicted , batch_output)
        correct_t += (predicted_t == batch_output_t).sum().item()
        total_t += batch_output_t.shape[0]*batch_output_t.shape[1]
        # print((predicted_t == batch_output_t).sum().item() , batch_output_t.shape[0]*batch_output_t.shape[1])
#         print(predicted_t , batch_output_t)

        # print(outputs.view(-1 , num_classes).shape , predicted_t.shape , batch_output_t.shape)
        # batch_output_t = batch_output_t.reshape(-1)
        # predicted_t = predicted_t.reshape(-1)
        # for i in range(len(batch_output_t)):
        #         if batch_output_t[i] != 0:
        #                 total_t+=1
        #                 if batch_output_t[i] == predicted_t[i]:
        #                         correct_t+=1

#     average_loss = total_loss / len(train_loader)
#     acc_train = correct_t / total_t
#     print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss:.4f} , acc : { acc_train :.4f} , { correct_t , total_t}')

    average_loss = total_loss / len(train_loader)
    accuracy = 100 * correct_t / (total_t)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss:.4f} , acc : { accuracy :.4f} , { correct_t , total_t}')

    # 검증 루프
    model.eval()
    correct = 0
    total = 0
    correct_v = 0
    total_v = 0

    with torch.no_grad():
        for batch_input, batch_output in valid_loader:
            batch_input = batch_input.to(device)
            batch_output = batch_output.to(device)
            outputs = model(batch_input.unsqueeze(1))
            predicted = outputs.argmax(dim=2)
    #         print(predicted , batch_output)
            correct += (predicted == batch_output).sum().item()
    #         print(batch_output.size(0))
            total += batch_output.shape[0]*batch_output.shape[1]


        #     batch_output = batch_output.reshape(-1)
        #     predicted = predicted.reshape(-1)
        #     for i in range(len(batch_output)):
        #         if batch_output[i] != 0:
        #                 total_v+=1
        #                 if batch_output[i] == predicted[i]:
        #                         correct_v+=1

        # acc_val = correct_v / total_v
        accuracy = 100 * correct / total

    print(f'Validation Accuracy: {accuracy:.2f}% , {correct , total}')
    if accuracy > acc_compare:
        acc_compare = accuracy
        torch.save(model.state_dict(), "LSTM_Head_All_230927_1.pt")

Epoch [1/100000], Loss: 0.9736 , acc : 63.5296 , (829061, 1305000)
Validation Accuracy: 69.19% , (226248, 327000)
Epoch [2/100000], Loss: 0.6971 , acc : 71.1025 , (927888, 1305000)
Validation Accuracy: 70.44% , (230336, 327000)
Epoch [3/100000], Loss: 0.6692 , acc : 71.4851 , (932881, 1305000)
Validation Accuracy: 70.99% , (232152, 327000)
Epoch [4/100000], Loss: 0.6636 , acc : 71.6261 , (934721, 1305000)
Validation Accuracy: 70.75% , (231345, 327000)
Epoch [5/100000], Loss: 0.6610 , acc : 71.6843 , (935480, 1305000)
Validation Accuracy: 71.37% , (233373, 327000)
Epoch [6/100000], Loss: 0.6593 , acc : 71.6808 , (935435, 1305000)
Validation Accuracy: 71.01% , (232191, 327000)
Epoch [7/100000], Loss: 0.6576 , acc : 71.7897 , (936855, 1305000)
Validation Accuracy: 71.36% , (233358, 327000)
Epoch [8/100000], Loss: 0.6571 , acc : 71.7785 , (936709, 1305000)
Validation Accuracy: 71.56% , (233996, 327000)
Epoch [9/100000], Loss: 0.6566 , acc : 71.8116 , (937142, 1305000)
Validation Accuracy: 

In [14]:
cor_t = batch_output.reshape(-1)
pre_t = predicted.reshape(-1)

In [15]:
t = 0
c = 0
t_list = []
c_list = []
for i in range(len(cor_t)):
    if cor_t[i] != 0:
        t+=1
        Y_val_np = cor_t[i].cpu().numpy()
        t_list.append(Y_val_np)
        pred_np = pre_t[i].cpu().numpy()
        c_list.append(pred_np)
        if cor_t[i] == pre_t[i]:
            c+=1

print('acc : ' , c/t)

# pred_np = pred.cpu().numpy()
# pred_np = pred_np.reshape(-1)
# Y_val_np = Y_val.cpu().numpy()
# Y_val_np = Y_val_np.reshape(-1)


acc :  1.0


In [ ]:
for i in range(len(t_list)):
    print(t_list[i] ,c_list[i] )

In [16]:
c_list.count(0)

0